In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps

taxi_output_data_file = "output_data/rides_june_2015.csv"
rideshare_output_data_file = "output_data/hv_rideshare_june_2019.csv"

#Import Manhattan zones df
manhattan_zones = pd.read_csv("../../../Desktop/NYC Taxi & FHV Data/manhattan_zones.csv")
manhattan_zones.head()

,LocationID,Borough,Zone,service_zone
0,4,Manhattan,Alphabet City,Yellow Zone
1,12,Manhattan,Battery Park,Yellow Zone
2,13,Manhattan,Battery Park City,Yellow Zone
3,24,Manhattan,Bloomingdale,Yellow Zone
4,41,Manhattan,Central Harlem,Boro Zone


# Yellow Taxi June 2015 Data

In [8]:
yellow_taxi_2015_06 = "../../../Desktop/NYC Taxi & FHV Data/yellow_tripdata_2015-06.csv"
yt_june_2015_df = pd.read_csv(yellow_taxi_2015_06)

In [6]:
clean_yt_june_2015_df= yt_june_2015_df.drop(columns=['VendorID','extra','store_and_fwd_flag','payment_type','mta_tax','tolls_amount','improvement_surcharge'])
clean_yt_june_2015_df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30


In [9]:
clean_yt_june_2015_df= yt_june_2015_df.drop(columns=['VendorID','extra','store_and_fwd_flag','payment_type','mta_tax','tolls_amount','improvement_surcharge'])
clean_yt_june_2015_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30
...,...,...,...,...,...,...,...,...,...,...,...,...
12324930,2015-06-30 21:54:20,2015-06-30 22:08:54,1,1.90,-73.924347,40.774315,1,-73.919617,40.755508,11.5,0.00,12.80
12324931,2015-06-30 21:54:20,2015-06-30 22:16:23,1,3.10,-73.951843,40.773781,1,-73.991394,40.759972,15.5,1.00,17.80
12324932,2015-06-30 21:54:21,2015-06-30 22:04:12,6,1.55,-73.961060,40.813545,1,-73.945053,40.808304,8.5,0.00,9.80
12324933,2015-06-30 21:54:21,2015-06-30 22:09:35,1,3.20,-73.984375,40.761627,1,-74.002548,40.730511,12.5,0.00,13.80


In [10]:
clean_yt_june_2015_df = clean_yt_june_2015_df.loc[clean_yt_june_2015_df["passenger_count"] != 0]

In [11]:
cut_dates = clean_yt_june_2015_df["tpep_pickup_datetime"].str.split(" ", n =1, expand = True)[0]
clean_yt_june_2015_df["tpep_pickup_datetime"] = cut_dates
clean_yt_june_2015_df.head()

/var/folders/w4/cz88rjkj0lg25jggxn27s8l40000gn/T/ipykernel_98102/2799628214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_yt_june_2015_df["tpep_pickup_datetime"] = cut_dates


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount
0,2015-06-02,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80
1,2015-06-02,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30
2,2015-06-02,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00
3,2015-06-02,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16
4,2015-06-02,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30


In [12]:
cut_month = clean_yt_june_2015_df['tpep_pickup_datetime'].str.split("-", expand = True)
#cut_month
clean_yt_june_2015_df["Month"] = cut_month[1].astype(int)
clean_yt_june_2015_df["Month"]
clean_yt_june_2015_df.head()

/var/folders/w4/cz88rjkj0lg25jggxn27s8l40000gn/T/ipykernel_98102/3668294939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_yt_june_2015_df["Month"] = cut_month[1].astype(int)


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount,Month
0,2015-06-02,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80,6
1,2015-06-02,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30,6
2,2015-06-02,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00,6
3,2015-06-02,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16,6
4,2015-06-02,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30,6


In [13]:
clean_yt_june_2015_df.groupby("Month").count()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount
Month,,,,,,,,,,,,
6,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381


In [15]:
manhattan_yt_june_2015_df = clean_yt_june_2015_df.loc[(clean_yt_june_2015_df["pickup_latitude"]<= 40.737052) & (clean_yt_june_2015_df["pickup_latitude"] >= 40.722288) & 
                               (clean_yt_june_2015_df["pickup_longitude"]<= -73.972107) & (clean_yt_june_2015_df["pickup_longitude"] >= -74.003929)]


manhattan_yt_june_2015_df = clean_yt_june_2015_df.loc[(clean_yt_june_2015_df["dropoff_latitude"]<= 40.737052) & (clean_yt_june_2015_df["dropoff_latitude"] >= 40.722288) & 
                               (clean_yt_june_2015_df["dropoff_longitude"]<= -73.972107) & (clean_yt_june_2015_df["dropoff_longitude"] >= -74.003929)]

manhattan_yt_june_2015_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount,Month
20,2015-06-02,2015-06-02 11:34:36,1,3.90,-73.949623,40.772671,1,-73.987640,40.735611,15.5,3.25,19.55,6
34,2015-06-02,2015-06-02 11:28:29,5,0.85,-74.006416,40.733501,1,-73.996292,40.725971,7.0,1.56,9.36,6
40,2015-06-02,2015-06-02 11:25:08,1,0.80,-73.995033,40.725391,1,-73.989548,40.735748,5.5,1.26,7.56,6
43,2015-06-02,2015-06-02 12:02:22,2,5.30,-73.955170,40.788544,1,-74.001060,40.736637,27.0,5.55,33.35,6
47,2015-06-02,2015-06-02 11:26:40,1,0.99,-73.997749,40.730190,1,-73.986282,40.734440,6.0,0.00,6.80,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12324898,2015-06-30,2015-06-30 22:00:04,1,1.20,-74.000076,40.730446,1,-73.987328,40.734974,6.0,2.15,9.45,6
12324902,2015-06-30,2015-06-30 22:04:18,1,1.50,-74.004044,40.738266,1,-73.986420,40.726517,8.0,0.00,9.30,6
12324912,2015-06-30,2015-06-30 22:03:21,1,0.95,-73.998016,40.737888,1,-73.982758,40.731483,7.5,0.00,8.80,6
12324927,2015-06-30,2015-06-30 22:11:42,2,3.50,-73.991806,40.759697,1,-73.981239,40.725273,14.5,3.15,18.95,6


# High Voume FHV June 2019 Data

In [17]:
hv_fhv_2019_06 = "../../../Desktop/NYC Taxi & FHV Data/clean_csvs/HighVol_june_2019.csv"
hv_fhv_june_2019_df = pd.read_csv(hv_fhv_2019_06)
hv_fhv_june_2019_df
clean_hv_fhv_june_2019_df = hv_fhv_june_2019_df.drop(columns=['Unnamed: 0'])
clean_hv_fhv_june_2019_df.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_date,pickup_time,dropoff_date,dropoff_time
0,HV0004,B02800,2019-06-01 00:09:57,2019-06-01 00:16:41,237,263,2019-06-01,00:09:57,2019-06-01,00:16:41
1,HV0003,B02869,2019-06-01 00:20:37,2019-06-01 00:37:38,45,263,2019-06-01,00:20:37,2019-06-01,00:37:38
2,HV0003,B02884,2019-06-01 00:30:42,2019-06-01 00:41:14,170,263,2019-06-01,00:30:42,2019-06-01,00:41:14
3,HV0003,B02764,2019-06-01 00:34:54,2019-06-01 00:39:28,263,236,2019-06-01,00:34:54,2019-06-01,00:39:28
4,HV0003,B02835,2019-06-01 00:34:46,2019-06-01 01:18:08,179,263,2019-06-01,00:34:46,2019-06-01,01:18:08


In [18]:
cut_dates = clean_hv_fhv_june_2019_df["pickup_datetime"].str.split(" ", n =1, expand = True)[0]
clean_hv_fhv_june_2019_df["pickup_datetime"] = cut_dates
clean_hv_fhv_june_2019_df

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_date,pickup_time,dropoff_date,dropoff_time
0,HV0004,B02800,2019-06-01,2019-06-01 00:16:41,237,263,2019-06-01,00:09:57,2019-06-01,00:16:41
1,HV0003,B02869,2019-06-01,2019-06-01 00:37:38,45,263,2019-06-01,00:20:37,2019-06-01,00:37:38
2,HV0003,B02884,2019-06-01,2019-06-01 00:41:14,170,263,2019-06-01,00:30:42,2019-06-01,00:41:14
3,HV0003,B02764,2019-06-01,2019-06-01 00:39:28,263,236,2019-06-01,00:34:54,2019-06-01,00:39:28
4,HV0003,B02835,2019-06-01,2019-06-01 01:18:08,179,263,2019-06-01,00:34:46,2019-06-01,01:18:08
...,...,...,...,...,...,...,...,...,...,...
249826,HV0005,B02510,2019-06-30,2019-06-30 23:30:13,255,263,2019-06-30,23:03:57,2019-06-30,23:30:13
249827,HV0004,B02800,2019-06-30,2019-06-30 23:52:03,263,179,2019-06-30,23:37:57,2019-06-30,23:52:03
249828,HV0003,B02764,2019-06-30,2019-06-30 23:34:53,263,265,2019-06-30,23:07:00,2019-06-30,23:34:53
249829,HV0005,B02510,2019-06-30,2019-06-30 23:32:39,263,162,2019-06-30,23:24:17,2019-06-30,23:32:39


In [19]:
cut_month = clean_hv_fhv_june_2019_df['pickup_datetime'].str.split("-", expand = True)
#cut_month
clean_hv_fhv_june_2019_df["Month"] = cut_month[1].astype(int)
clean_hv_fhv_june_2019_df["Month"]
clean_hv_fhv_june_2019_df.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_date,pickup_time,dropoff_date,dropoff_time,Month
0,HV0004,B02800,2019-06-01,2019-06-01 00:16:41,237,263,2019-06-01,00:09:57,2019-06-01,00:16:41,6
1,HV0003,B02869,2019-06-01,2019-06-01 00:37:38,45,263,2019-06-01,00:20:37,2019-06-01,00:37:38,6
2,HV0003,B02884,2019-06-01,2019-06-01 00:41:14,170,263,2019-06-01,00:30:42,2019-06-01,00:41:14,6
3,HV0003,B02764,2019-06-01,2019-06-01 00:39:28,263,236,2019-06-01,00:34:54,2019-06-01,00:39:28,6
4,HV0003,B02835,2019-06-01,2019-06-01 01:18:08,179,263,2019-06-01,00:34:46,2019-06-01,01:18:08,6


In [20]:
manhattan_area = manhattan_zones["LocationID"]

for zone in manhattan_area:
    manhattan_hv_fhv_june_2019 = clean_hv_fhv_june_2019_df.loc[(clean_hv_fhv_june_2019_df["PULocationID"] == zone) | 
                                                                (clean_hv_fhv_june_2019_df["DOLocationID"] == zone)
                                                                  ]


manhattan_hv_fhv_june_2019

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_date,pickup_time,dropoff_date,dropoff_time,Month
0,HV0004,B02800,2019-06-01,2019-06-01 00:16:41,237,263,2019-06-01,00:09:57,2019-06-01,00:16:41,6
1,HV0003,B02869,2019-06-01,2019-06-01 00:37:38,45,263,2019-06-01,00:20:37,2019-06-01,00:37:38,6
2,HV0003,B02884,2019-06-01,2019-06-01 00:41:14,170,263,2019-06-01,00:30:42,2019-06-01,00:41:14,6
3,HV0003,B02764,2019-06-01,2019-06-01 00:39:28,263,236,2019-06-01,00:34:54,2019-06-01,00:39:28,6
4,HV0003,B02835,2019-06-01,2019-06-01 01:18:08,179,263,2019-06-01,00:34:46,2019-06-01,01:18:08,6
...,...,...,...,...,...,...,...,...,...,...,...
249826,HV0005,B02510,2019-06-30,2019-06-30 23:30:13,255,263,2019-06-30,23:03:57,2019-06-30,23:30:13,6
249827,HV0004,B02800,2019-06-30,2019-06-30 23:52:03,263,179,2019-06-30,23:37:57,2019-06-30,23:52:03,6
249828,HV0003,B02764,2019-06-30,2019-06-30 23:34:53,263,265,2019-06-30,23:07:00,2019-06-30,23:34:53,6
249829,HV0005,B02510,2019-06-30,2019-06-30 23:32:39,263,162,2019-06-30,23:24:17,2019-06-30,23:32:39,6
